In [1]:
# download the dataset, extract, and remove compressed version
!kaggle datasets download -d usdot/flight-delays -f "flights.csv"
!unzip flights.csv.zip
!rm flights.csv.zip

/bin/bash: kaggle: command not found
unzip:  cannot find or open flights.csv.zip, flights.csv.zip.zip or flights.csv.zip.ZIP.
rm: cannot remove 'flights.csv.zip': No such file or directory


In [2]:
# initlize pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("airline-delay-analysis").getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3435m
22/05/10 18:39:44 WARN Utils: Your hostname, fawazalesay-pysparkairl-367azzi4cid resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface ceth0)
22/05/10 18:39:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 18:39:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

In [22]:
# Read the data into a dataframe and print the schema
df = spark.read.csv("flights.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [23]:
from pyspark.sql.functions import *

# Add a column "delayed" which is 0 if DEPARTURE_DELAY is 0, 1 if DEPARTURE_DELAY is greater than 0 and -1 if DEPARTURE_DELAY is less than 0
df = df.withColumn("DELAYED", when(df["DEPARTURE_DELAY"] == 0, 0).otherwise(when(df["DEPARTURE_DELAY"] > 0, 1).otherwise(2)))

In [6]:
# Our Schema
# |-- YEAR: integer (nullable = true) - The year of the flight
# |-- MONTH: integer (nullable = true) - The month of the flight
# |-- DAY: integer (nullable = true) - The day of the flight
# |-- DAY_OF_WEEK: integer (nullable = true) - The day of the week of the flight
# |-- AIRLINE: string (nullable = true) - The airline code of the flight
# |-- FLIGHT_NUMBER: integer (nullable = true) - The flight number of the flight
# |-- TAIL_NUMBER: string (nullable = true) - the tail number of the aircraft
# |-- ORIGIN_AIRPORT: string (nullable = true) - The origin airport code of the flight
# |-- DESTINATION_AIRPORT: string (nullable = true) - The destination airport code of the flight
# |-- SCHEDULED_DEPARTURE: integer (nullable = true) - The scheduled departure time of the flight
# |-- DEPARTURE_TIME: integer (nullable = true) - The actual departure time of the flight
# |-- DEPARTURE_DELAY: integer (nullable = true) - The departure delay of the flight
# |-- TAXI_OUT: integer (nullable = true) - The taxi out time of the flight
# |-- WHEELS_OFF: integer (nullable = true) - The wheels off time of the flight
# |-- SCHEDULED_TIME: integer (nullable = true) - The scheduled time of the flight
# |-- ELAPSED_TIME: integer (nullable = true) - The elapsed time of the flight
# |-- AIR_TIME: integer (nullable = true) - The air time of the flight
# |-- DISTANCE: integer (nullable = true) - The distance between the origin and destination airports
# |-- WHEELS_ON: integer (nullable = true) - The wheels on time of the flight
# |-- TAXI_IN: integer (nullable = true) - The taxi in time of the flight
# |-- SCHEDULED_ARRIVAL: integer (nullable = true) - The scheduled arrival time of the flight
# |-- ARRIVAL_TIME: integer (nullable = true) - The actual arrival time of the flight
# |-- ARRIVAL_DELAY: integer (nullable = true) - The arrival delay of the flight
# |-- DIVERTED: integer (nullable = true) - Whether the flight was diverted or not
# |-- CANCELLED: integer (nullable = true) - Whether the flight was cancelled or not
# |-- CANCELLATION_REASON: string (nullable = true) - The reason the flight was cancelled
# |-- AIR_SYSTEM_DELAY: integer (nullable = true) - The air system delay of the flight
# |-- SECURITY_DELAY: integer (nullable = true) - The security delay of the flight
# |-- AIRLINE_DELAY: integer (nullable = true) - The airline delay of the flight
# |-- LATE_AIRCRAFT_DELAY: integer (nullable = true) - The late aircraft delay of the flight 
# |-- WEATHER_DELAY: integer (nullable = true) - The weather delay of the flight


In [26]:
# Preprocess the data
# Drop the columns that we don't need or that we cannot predict before the flight

# Drop the following columns
# |-- YEAR: integer (nullable = true) - Not needed since all flights are from the same year
# |-- FLIGHT_NUMBER: integer (nullable = true) - Not needed since all planes have a unique flight number
# |-- TAIL_NUMBER: string (nullable = true) - Not needed since all planes have a unique tail number
# |-- DEPARTURE_TIME: integer (nullable = true) - Cannot know this before the flight
# |-- DEPARTURE_DELAY: integer (nullable = true) - Cannot know this before the flight
# |-- TAXI_OUT: integer (nullable = true) - Cannot know this before the flight
# |-- WHEELS_OFF: integer (nullable = true) - Cannot know this before the flight
# |-- ELAPSED_TIME: integer (nullable = true) - Cannot know this before the flight
# |-- AIR_TIME: integer (nullable = true) - Cannot know this before the flight
# |-- WHEELS_ON: integer (nullable = true) - Cannot know this before the flight
# |-- TAXI_IN: integer (nullable = true) - Cannot know this before the flight
# |-- ARRIVAL_TIME: integer (nullable = true) - Cannot know this before the flight
# |-- ARRIVAL_DELAY: integer (nullable = true) - Not needed since we will only focus on the departure delay
# |-- DIVERTED: integer (nullable = true) - Cannot know this before the flight
# |-- CANCELLED: integer (nullable = true) - Cannot know this before the flight
# |-- CANCELLATION_REASON: string (nullable = true) - Cannot know this before the flight
# |-- AIR_SYSTEM_DELAY: integer (nullable = true) - Cannot know this before the flight
# |-- SECURITY_DELAY: integer (nullable = true) - Cannot know this before the flight
# |-- AIRLINE_DELAY: integer (nullable = true) - Cannot know this before the flight
# |-- LATE_AIRCRAFT_DELAY: integer (nullable = true) - Cannot know this before the flight
# |-- WEATHER_DELAY: integer (nullable = true) - Cannot know this before the flight

df = df.drop("YEAR", "FLIGHT_NUMBER", "TAIL_NUMBER", "DEPARTURE_TIME", "DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "ELAPSED_TIME", "AIR_TIME", "WHEELS_ON", "TAXI_IN", "ARRIVAL_TIME", "ARRIVAL_DELAY", "DIVERTED", "CANCELLED", "CANCELLATION_REASON", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY", "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY")

# Our new schema
# |-- MONTH: integer (nullable = true)
# |-- DAY: integer (nullable = true)
# |-- DAY_OF_WEEK: integer (nullable = true)
# |-- AIRLINE: string (nullable = true)
# |-- ORIGIN_AIRPORT: string (nullable = true)
# |-- DESTINATION_AIRPORT: string (nullable = true)
# |-- SCHEDULED_DEPARTURE: integer (nullable = true)
# |-- SCHEDULED_TIME: integer (nullable = true)
# |-- DISTANCE: integer (nullable = true)
# |-- SCHEDULED_ARRIVAL: integer (nullable = true)
# |-- DELAYED: integer (nullable = false)
df.printSchema()

root
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- DELAYED: integer (nullable = false)



In [31]:
# Remove the rows that have null values
df = df.dropna()

# Print the number of rows
print(df.count())

5819073


In [37]:
# Use a Decision Tree Classifier to predict the delay
# Split the data into training and testing sets
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import StringIndexer

train, test = df.randomSplit([0.7, 0.3])

# Encode the categorical features using StringIndexer
indexer = StringIndexer(inputCols=["AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], outputCols=["AIRLINE_INDEX", "ORIGIN_AIRPORT_INDEX", "DESTINATION_AIRPORT_INDEX"])

# Create the assembler
assembler = VectorAssembler(inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "AIRLINE_INDEX", "ORIGIN_AIRPORT_INDEX", "DESTINATION_AIRPORT_INDEX", "SCHEDULED_DEPARTURE", "SCHEDULED_TIME", "DISTANCE", "SCHEDULED_ARRIVAL"], outputCol="features")

# Get the maximum number of all categorical features in the dataframe
max_categories = df.select("AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT").distinct().count()

# Create the classifier
dt = DecisionTreeClassifier(labelCol="DELAYED", featuresCol="features", maxBins=max_categories)

# Create the pipeline
pipeline = Pipeline(stages=[indexer, assembler, dt])

# Create the grid
# paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [2, 3, 4, 5, 6, 7, 8, 9, 10]).addGrid(dt.maxBins, [650, 750, 850, 950, 1050, 1150, 1250]).build()

# Create the cross validator
# cv = CrossValidator(estimator=pipeline, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid, numFolds=3)


# Create the train validation split with the param grid
# tvs = TrainValidationSplit(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=MulticlassClassificationEvaluator(), trainRatio=0.8)

# Create the train validation split without the param grid
# tvs = TrainValidationSplit(estimator=pipeline, evaluator=MulticlassClassificationEvaluator(), trainRatio=0.8)

# Train the model using the pipeline
model_basic = pipeline.fit(train)

# Train the model using TrainValidationSplit
# model = tvs.fit(train)

# Train the model using CrossValidator
# model = cv.fit(train)

# Evaluate the models
predictions_basic = model_basic.transform(test)

# predictions_with_validation = tvs.fit(train).transform(test)



# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="DELAYED", predictionCol="prediction", metricName="accuracy")

# Print the accuracy
print("Accuracy (No Train Validation)", evaluator.evaluate(predictions_basic))